In [366]:
from sklearn import model_selection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import *
import math
%matplotlib inline 
%config InlineBackend.figure_format="retina" 

In [367]:
# 数据处理
continus={}
def ReadData(pathname):
    data=pd.read_csv(pathname)
    data.loc[data['好瓜']=='是', '好瓜'] = 1
    data.loc[data['好瓜']=='否', '好瓜'] = 0
    
    
    data = data.iloc[:,1:]
    
    
    attribute=data.columns
    for i in attribute[:-1]:
        m=0
        if(i!="密度" and i!="含糖率"):
            for j in data[i].unique():
                data.loc[data[i]==j, i]=m
                m+=1
        else:
            for j in data[i].unique():
                continus[i,str(j)]=m
                m+=1
            
    
    label=data['好瓜'].unique()
    
    #print(data)
    #print(continus)
     
    return data,attribute,label

In [424]:
def calContProb(data,feature,attribute):
    if(feature!="密度" and feature!="含糖率"):
        return calContProbDis(data,feature,attribute)
    else:
        return
    
def calContProbDis(data,feature,attribute):
    
    ni=len(data[feature].unique())
    prob=np.zeros((len(data['好瓜'].unique())))
    
    for i in data['好瓜'].unique():
        nc=len(data)+ni
        prob[i]=(len(data[data['好瓜']==i][data[feature]==0])+1)/nc
        #temp=1
        for k in attribute:
            if (k!="密度" and k!="含糖率"):
                prob[i]*=calContProbDis_xij(data,i,0,feature,k)
                #temp*=calContProbDis_xij(data,i,0,feature,k)
            else:
                break
            #print(feature,0,k,0,temp)
    return prob

# 不同于拉普拉斯法，注意涉及xj,直接求乘积即可
# 只需要针对xi=0和xj=0（也就是测试1计算即可）

def calContProbDis_xij(data,c,xi,feature1,feature2):
    
    xj=0
    nc=len(data[data['好瓜']==c][data[feature1]==xi])+len(data[feature2].unique())
    prob=(len(data[data['好瓜']==c][data[feature1]==xi][data[feature2]==xj])+1)/nc
    
    #if(feature1=="敲声" and feature2=="脐部" and xi==0 and xj==0):
    #    print(c,len(data[data['好瓜']==c][data[feature1]==xi][data[feature2]==xj]),len(data[data['好瓜']==c][data[feature1]==xi]),prob)
    return prob

In [426]:
def trainAODE(data,attribute):
    prob={}
    for i in attribute: 
        if i!='好瓜':
            prob[i]=calContProb(data, i, attribute)
    #print(prob)        
    return prob


In [430]:
def infer(model,label,attribute,labelData):
    res = {}
    for i in labels:
        res[i] = 0
        
        for j in attribute:

            res[i]+=model[j][i]
            #print(j,i,model[j][i][0],res[i])

    return res

In [431]:
def judge(res):
    if(res[1]>=res[0]):
        print('判断为好瓜')
    else:
        print('判断为坏瓜')

In [433]:

if __name__ == '__main__':
    data,attribute,labels=ReadData('../../data/watermelon_3.csv')
    model = trainAODE(data,attribute)
    #print(model)
    result = infer(model, labels, attribute[:-3],data['好瓜'])
    judge(result)
    print(result)
    

判断为好瓜
{1: 0.09695613977659626, 0: 0.003268716304243502}


/Users/wangxu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/wangxu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/wangxu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
